In [27]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
from fiftyone.core.labels import Detection
from PIL import Image

import torch
import torch.nn as nn
import torch.utils
import torch.utils.data

import torchvision
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.models.detection.mask_rcnn import maskrcnn_resnet50_fpn, MaskRCNNPredictor
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import matplotlib.pyplot as plt
import numpy as np

import os
import importlib
import sys
from pathlib import Path
from datetime import datetime

import utils
import train
from train import train_one_epoch, evaluate
import transforms as T

here = os.getcwd()
sys.path.append(os.path.join(here, '..'))
import DatasetLoaders
importlib.reload(DatasetLoaders)
from DatasetLoaders import HRSIDSegmentationDataset
import MaskRCNN_model_wrapper
from MaskRCNN_model_wrapper import MaskRCNNWrapper
importlib.reload(MaskRCNN_model_wrapper)

<module 'MaskRCNN_model_wrapper' from '/home/k3vinli/ENEE439/Capstone/models/mask_rcnn/MaskRCNN_model_wrapper.py'>

In [2]:
# get path to directory above
p = Path()
top_dir = p.absolute().parents[1]
top_dir

PosixPath('/home/k3vinli/ENEE439/Capstone')

In [13]:
name = "SSDD_train"
if name in fo.list_datasets():
    dataset_train = fo.load_dataset(name)
else:
    dataset_path = top_dir / "Datasets" / "Official-SSDD-OPEN" / "BBox_RBox_PSeg_SSDD" / "coco_style" / "images" / "train"
    labels_path = top_dir / "Datasets" / "Official-SSDD-OPEN" / "BBox_RBox_PSeg_SSDD" / "coco_style" / "annotations" / "train.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_train = fo.Dataset.from_dir(
        data_path=dataset_path,
        labels_path=labels_path,
        dataset_type=dataset_type,
        name=name
    )

 100% |█████████████████| 928/928 [3.4s elapsed, 0s remaining, 273.1 samples/s]      


In [12]:
# dataset_train.delete()

In [16]:
name = "SSDD_test"
if name in fo.list_datasets():
    dataset_test = fo.load_dataset(name)
else:
    dataset_path = top_dir / "Datasets" / "Official-SSDD-OPEN" / "BBox_RBox_PSeg_SSDD" / "coco_style" / "images" / "test"
    labels_path = top_dir / "Datasets" / "Official-SSDD-OPEN" / "BBox_RBox_PSeg_SSDD" / "coco_style" / "annotations" / "test.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_test = fo.Dataset.from_dir(
        data_path=dataset_path,
        labels_path=labels_path,
        dataset_type=dataset_type,
        name=name
    )

 100% |█████████████████| 232/232 [1.0s elapsed, 0s remaining, 231.9 samples/s]         


In [15]:
# dataset_test.delete()

In [17]:
dataset_train.compute_metadata()
dataset_test.compute_metadata()

In [18]:
paths = dataset_train.values("filepath")
sample = dataset_train[paths[1]]
sample.metadata

<ImageMetadata: {
    'size_bytes': None,
    'mime_type': None,
    'width': 500,
    'height': 403,
    'num_channels': None,
}>

In [19]:
dataset_train

Name:        SSDD_train
Media type:  image
Num samples: 928
Persistent:  False
Tags:        []
Sample fields:
    id:            fiftyone.core.fields.ObjectIdField
    filepath:      fiftyone.core.fields.StringField
    tags:          fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

In [20]:
session = fo.launch_app()

In [28]:
maskrcnn = MaskRCNNWrapper(2, True)

In [29]:
HRSID_train = HRSIDSegmentationDataset(dataset_train, transforms=maskrcnn.transforms)
HRSID_test = HRSIDSegmentationDataset(dataset_test, transforms=maskrcnn.transforms)

In [ ]:
def do_training(model, torch_dataset, torch_dataset_test, num_epochs=4, print_freq=10):
    data_loader = torch.utils.data.DataLoader(
        torch_dataset, batch_size=4, shuffle=False, num_workers=8,
        collate_fn=utils.collate_fn
    )
    data_loader_test = torch.utils.data.DataLoader(
        torch_dataset_test, batch_size=2, shuffle=False, num_workers=8,
        collate_fn=utils.collate_fn
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using {device}")
    model.to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.5, weight_decay=0.0005)

    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                    step_size=20,
                                                    gamma=0.1)
    for epoch in range(num_epochs):
        train_one_epoch(model=model,
                        optimizer=optimizer, 
                        data_loader=data_loader,
                        device=device, 
                        epoch=epoch, 
                        print_freq=print_freq)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        lr_scheduler.step()
        evaluate(model=model, data_loader=data_loader_test, device=device)

In [30]:
do_training(maskrcnn.model, HRSID_train, HRSID_test, num_epochs=100)

Using cuda


TypeError: 'module' object is not callable